# 1. Identifying the features
#### Use RFM scores for each customer ID for feature set. In order to implement this correctly, we should divide our dataset. We will select a period of 3 months' worth of data, calculate the RFM scores, and then utilize them to predict the following 6 months. Therefore, our first step is to create two dataframes and add the RFM scores to them.'

# 2. Importing necessary libraries and packages

In [11]:
#import libraries
from __future__ import division

from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
# from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# from sklearn.cluster import KMeans


import plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

# import xgboost as xgb
# from sklearn.model_selection import KFold, cross_val_score, train_test_split

# import xgboost as xgb

In [12]:
data = pd.read_csv('../data/customer_segmentation.csv', encoding='cp1252')

In [ ]:
#initate plotly
pyoff.init_notebook_mode()

#read data from csv and redo the data work we done before
data.head()